In [1]:
# Section 1 - Imports
# -------------------

# imports
import datetime
import attotime
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import math
from os.path import join

# DSO modules
from dsoclasses.doris.algorithms import beacon_nominal_frequency
from dsoclasses.rinex.doris.rinex import DorisRinex
from dsoclasses.time.pyattotime import at2pt, fsec2asec
from dsoclasses.orbits import sp3c, interpolator
from dsoclasses.geodesy import transformations
from dsoclasses.gnss import systems as gs
from dsoclasses.gnss import algorithms as alg
from dsoclasses.troposphere.vmf3 import SiteVmf3
from dsoclasses.sinex import sinex

In [2]:
# Section 2.1 – Define Files needed for analysis
# -----------------------------------------------

data_path = "/home/xanthos/Software/AcademicSoftware/data"
drinex = join(data_path, "s6arx24001.001")
dpod = join(data_path, "dpod2020_041.snx")
dpod_freq_corr = join(data_path, "dpod2020_041_freq_corr.txt")
dsp3 = join(data_path, "ssas6a20.b23357.e24001.DG_.sp3.001")
vmf3_data = join(data_path, "y2024.vmf3_d")

In [3]:
# Section 2.2 - Load Orbit Data + Initialize Interpolator
# -------------------------------------------------------

intrp = interpolator.Sp3Interpolator.from_sp3(dsp3, ['L'], interval_in_sec=310, min_data_pts=10, itype='Barycentric')

In [4]:
# Section 2.3 - Intitialize DorisRinex and select a beacon (Dionysos/DIOB)
# ------------------------------------------------------------------------

rnx = DorisRinex(drinex)

site_name = 'DIOB'
result = sinex.extract_sinex_coordinates(dpod, [site_name], rnx.time_first_obs, True, dpod_freq_corr)
diob_xyz = np.array([result['DIOB']['X'], result['DIOB']['Y'], result['DIOB']['Z']])

In [5]:
# Section 2.4 - Initialize VMF3 from (DORIS) site data
# ----------------------------------------------------
vmf = SiteVmf3(vmf3_data, [site_name])

In [8]:
# Section 3.1 - Prerequisites & helper functions
# ----------------------------------------------

# Ionosphere-free LC for phase observables
def getL3fun(nS1, nU2):
    gamma_sqrt = nS1 / nU2
    def l3(s1, u2):
        return s1 + (s1-gamma_sqrt * u2) / (gamma_sqrt*gamma_sqrt-1.)
    return l3

# For DIOB ...
s1, u2 = beacon_nominal_frequency(rnx.kfactor(site_name))
l3 = getL3fun(s1, u2)

In [9]:
# Section 3.2 - Initialize variables
# ----------------------------------

tprev   = None # previous (to current) observation epoch
L3_prev = None # previous L3 observation

In [ ]:
# Section 4 – Preprocess Doppler observations (filter valid ones for LS)
# ----------------------------------------------------------------------

# for every block in the RINEX file
for block in rnx:
    # for every beacon in the block
    for beacon, data in block:
        # match DIOB
        if beacon == rnx.name2id(site_name):
            
            # use the block, RINEX provided clock correction to get to (approximate) TAI
            # date TAI = epoch + receiver clock offset
            tai = block.t() + attotime.attotimedelta(nanoseconds=block.clock_offset() * 1e9)

            # Satellite position at signal emission time
            satx, saty, satz, _, _ = alg.sat_at_emission_time(diob_xyz, tai, intrp, 'L40')
            rsat = np.array([satx, saty, satz])

            # Elevation and tropospheric delay
            _, _, el = transformations.azele(rsat, diob_xyz)
            if el >= CUTOFF_ANGLE:
                dT = vmf.tropo_dealy(site_id, lat, lon, el, at2pt(t2))
    
                # Phase measurements L3 combination
                L3 = l3(data['L1']['value'], data['L2']['value'])
    
                # apply frequency offset - get true receiver frequency
                frT = s1 * (1 + data['F']['value'] * 1e-11)
    
                # if we have a previous measurement, within 10 sec ...
                if tprev is not None and (tai - tprev).total_seconds() <= 10.:
                    # observed range-rate
                    orr = (((L3 - L3_prev) / (float((t2-t1).total_nanoseconds()) * 1e-9)) + frT - s1) * (299792458.0 / feN3)
                    
                    t.append(at2pt(tai))
                    el_valid.append(np.degrees(el))
                    dT_list.append(dT)
                    raw_obs.append({
                        'rsat1': np.array([satx_prev, saty_prev, satz_prev]),
                        'rsat2': rsat2,
                        'el': el_prev + (el-el_prev)/2.,
                        'dt': (float((t2-t1).total_nanoseconds()) * 1e-9),
                        'v_obs': orr,
                        'dT': (dT2 - dT) # if dT2 is not None else 0.0,
                    })
                    
            # prepare for next measurement
                dT2 = dT
                t1 = t2
                L1 = L3
                satx_prev, saty_prev, satz_prev = satx, saty, satz
                el_prev = el
print(f"Number of Doppler observations extracted: {len(raw_obs)}")